##Hands-on Lab: Build an Interactive Dashboard with Plotly Dash

In this lab, you will be building a Plotly Dash application for users to perform interactive visual analytics on SpaceX launch data in
real-time.

This dashboard application contains input components such as a dropdown list and a range slider to
interact with a pie chart and a scatter point chart. You will be guided to build this dashboard application via the following tasks:

    TASK 1: Add a Launch Site Drop-down Input Component
    TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown
    TASK 3: Add a Range Slider to Select Payload
    TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot

In [ ]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [ ]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [ ]:
# Create a dash application
app = dash.Dash(__name__)


In [ ]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                                options=[
                                                    {'label': 'All Sites', 'value': 'ALL'},
                                                    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                    {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}
                                                ],
                                                value='ALL',
                                                placeholder="Select a launch site",
                                                searchable=True
                                                ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                # min_payload = spacex_df['Payload Mass (kg)'].min()
                                # max_payload = spacex_df['Payload Mass (kg)'].max()
                                dcc.RangeSlider(id='payload-slider',
                                                    min = 0,
                                                    max = 10000,
                                                    step = 1000,
                                                    marks={0: '0', 100: '100'},
                                                    value=[0, 9600]
                                                    ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])



In [ ]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class',
        names='Launch Site',
        title='Distribution of successes per launch site')
        return fig
    else:
        site_df = spacex_df.loc[spacex_df['Launch Site'] == entered_site]
        pos = site_df['class'].sum()
        neg = len(site_df) - pos
        fig = px.pie(site_df, values=[pos,neg],
        names=['success', 'failure'],
        title="Successes vs failures at "+entered_site)
        return fig



In [ ]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'),
              Input(component_id='payload-slider', component_property='value'))
def get_scatter_plot(site, payload):
    clean_df = spacex_df.loc[spacex_df['Payload Mass (kg)'].between(payload[0], payload[1]) ]
    if site == 'ALL':
        fig = px.scatter(clean_df, x = 'Payload Mass (kg)', y = 'class',
        color = "Booster Version Category",
        title='Distribution of payload mass across outcomes')
        fig.update_layout(autosize=False)
        return fig
    else:
        site_df = clean_df.loc[clean_df['Launch Site'] == site]
        # pos = site_df['class'].sum()
        # neg = len(site_df) - pos
        fig = px.scatter(site_df, x = 'Payload Mass (kg)', y = 'class',
        color = "Booster Version Category",
        title="Distribution of payload mass across outcomes at "+site)
        fig.update_layout(autosize=False)
        return fig



In [ ]:
# Run the app
if __name__ == '__main__':
    app.run(debug=True)
